In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import *
class Rec_RNN(nn.Module):
    def __init__(self,inpput_size,hidden_size,output_size):
        super().__init__()
        self.hidden_size=hidden_size
        self.input_size=inpput_size
        self.output_size=output_size

        self.rnn=nn.GRU(input_size=inpput_size,hidden_size=hidden_size,num_layers=1)
        self.relu=nn.ReLU()
        self.linear=nn.Linear(hidden_size,output_size)
    def init_state(self,batch_size):
        self.state=torch.zeros(1,batch_size,self.hidden_size)
        return self.state
    def forward(self,x):
        x=x.permute(1,0,2)
        y_hat,state=self.rnn(x)
        y_hat=self.linear(y_hat.reshape((-1,y_hat.shape[-1])))
        return y_hat,state

In [2]:
from torchsummary import summary
# from project.model.RNN import *
net=Rec_RNN(28*28,1024,10)
summary(net,(1,28*28),batch_size=2,device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
               GRU-1  [[-1, 2, 1024], [-1, 2, 1024]]               0
            Linear-2                    [2, 10]          10,250
Total params: 10,250
Trainable params: 10,250
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 32.00
Params size (MB): 0.04
Estimated Total Size (MB): 32.05
----------------------------------------------------------------


In [3]:
def setProjectPath(projectPath):
    import os
    import sys
    root=os.path.abspath(projectPath)
    print(root)
    sys.path.append(root)
    os.environ["TORCH_HOME"]=r"E:\Data\torch-model"
    import warnings  
    warnings.filterwarnings("ignore", category=RuntimeWarning, module="mne")


In [4]:
projectPath=r'D:\project_meta\NNproject\NNI'
setProjectPath(projectPath)
import mne
datasetFolder=r'D:\dataset\driver_dataset\DROZY\DROZY'
outputFolder=r'D:\project_meta\NNproject\NNI\output'
from project.dataprocess.FolderTree import *
datasetDict=getDataPath(datasetFolder)
outputDict=getOutPath(outputFolder)
import pprint
pprint.pprint(datasetDict)
pprint.pprint(outputDict)

D:\project_meta\NNproject\NNI
{'annotation_auto_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-auto',
 'annotation_manual_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-manual',
 'kinect_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\kinect-intrinsics.yaml',
 'kss_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\KSS.txt',
 'psg_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\psg',
 'pvt_rt_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\pvt-rt',
 'videos_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\videos_i8'}
{'ECG_filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\ECG',
 'ECG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\ECG_table.csv',
 'ECG_uniformfilted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\ECG',
 'EEG_filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\EEG',
 'EEG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\EEG_table.csv',
 'EEG_unifor

# 模型

In [12]:
from project.model.CNN1D import *
from torchvision.datasets import *
from torchvision.transforms import *
import torchvision
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
])
MNISTDataset=FashionMNIST(root=r'D:/dataset',transform=train_transforms)


In [32]:
from torchsummary import summary
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from lightning.pytorch.core.module import LightningModule
from torch.utils.data import DataLoader
from torchmetrics.functional import accuracy
class ParityModuleRNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.flatten=nn.Flatten()
        self.rnn = nn.LSTM(28*28, 512, batch_first=True)
        self.linear_out = nn.Linear(in_features=512, out_features=10)
        self._loss = []  # needed for checking if the loss is the same as vanilla torch

    def forward(self, x):
        x=self.flatten(x)
        seq, last = self.rnn(x)
        y=self.linear_out(seq)
        
        # print(y.shape)
        return y

    def training_step(self, batch):
        x, y = batch
        # print(y.shape)
        y_hat = self(x)
        # print(y_hat.shape)
        loss = F.cross_entropy(y_hat, y)
        acc=accuracy(y_hat,y,task='multiclass',num_classes=10)
        self.log("train_acc", acc, on_step=True, on_epoch=True)
        self.log("trainloss", loss, on_step=True, on_epoch=True)
        self._loss.append(loss.item())
        return {"loss": loss}

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)
    def validation_step(self,batch, *args: Any, **kwargs: Any) :
        x,y=batch
        y_hat=self(x)
        loss=F.cross_entropy(y_hat, y)
        acc=accuracy(y_hat,y,task='multiclass',num_classes=10)
        self.log("test_acc", acc, on_step=False, on_epoch=True)
        self.log("testloss", loss, on_step=False, on_epoch=True)
        return {'loss':loss}
        


plnet=ParityModuleRNN()
plnet(torch.rand([2,1,28*28])).shape


torch.Size([2, 10])

In [33]:
from torch.utils.data import random_split
import pytorch_lightning as pl
train, val = random_split(MNISTDataset, [55000, 5000])
trainer = pl.Trainer()
trainer.progress_bar_refresh_rate = 10
trainer.fit(plnet, DataLoader(train,batch_size=64),DataLoader(val,batch_size=64))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type    | Params
---------------------------------------
0 | flatten    | Flatten | 0     
1 | rnn        | LSTM    | 2.7 M 
2 | linear_out | Linear  | 5.1 K 
---------------------------------------
2.7 M     Trainable params
0         Non-trainable params
2.7 M     Total params
10.654    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\call.py:52: Detected KeyboardInterrupt, attempting graceful shutdown...
